# Prédiction

## Encodage

- Utilisation du OneHotEncoding
- Scaling de nos données continues
- Passage au log pour les données skewées

## Sélection

# Sources d'apprentissage externes

- [Etre plus à l'aise avec l'IA de manière générale](https://www.youtube.com/watch?v=P6kSc3qVph0&list=PLO_fdPEVlfKoHQ3Ua2NtDL4nmynQC8YiS)
- Shap : [YT](https://www.youtube.com/watch?v=IqT551LjKHw) / [Kaggle](https://www.kaggle.com/code/prashant111/explain-your-model-predictions-with-shapley-values)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = 50
pd.options.display.max_rows = 100
np.set_printoptions(linewidth=180)
sns.set_style("dark")
sns.axes_style("darkgrid")

data = pd.read_csv("cleaned.csv")
data.columns

# Itération 1


In [ ]:
df = data.copy()
df = df.drop(columns="ENERGYSTARScore")

In [ ]:
df.dtypes.value_counts()

In [ ]:
df.head()

On peut déjà sélectionner les features à OneHotEncoder.

On traitera les PropertyUseType séparemment puisqu'il y a un lien entre les 3 features UseTypes.

In [ ]:
use_types = ["LargestPropertyUseType", "SecondLargestPropertyUseType", "ThirdLargestPropertyUseType"]
use_types_gfa = [x+"GFA" for x in use_types]
unique_use_types = np.array(list(set(df[use_types].to_numpy().ravel())))
unique_use_types

On souhaite avoir un dataframe avec en colonnes les valeurs possibles d'un use_type et en donnée un 1 ou un 0 suivant si ce type existe dans LargestPropertyUseType, SecondLargestPropertyUseType, ThirdLargestPropertyUseType.

Mais on a un problème. On aura un tableau du style

| LargestPropertyUseTypeGFA |SecondLargestPropertyUseTypeGFA |ThirdLargestPropertyUseTypeGFA | Adult Education | Automobile Dealership | Bank Branch | Bar/Nightclub | ... |
|-|-|-|-|-|-|-| - |
|0.45|0.10|0.05|0|1|1|0| ... |

Comment dans notre example on peut retrouver que le LargestPropertyUseType de 0.45 fait référence à la Bank Branch ? Aucun moyen si on reste sur cette structure.

La solution est de ne pas voir les données en OneHotEncode. Ainsi on aurait ce résultat :

| Adult Education | Automobile Dealership | Bank Branch | Bar/Nightclub | ... |
|-|-|-|-| - |
|0|0.10|0.45|0| ... |

Ici on conserve l'information du GFA mais on la met directement dans la colonne représentant la valeur elle même. Ainsi le modèle pourra pondérer en fonction du bon UseType.

Cela nous permet également de retirer nos features de GFA ce qui réduit la dimension d'entrée au modèle.

In [ ]:
def encode_use_types(df, use_types_gfa=use_types_gfa):
    use_types_gfa_df = pd.DataFrame()

    df[use_types_gfa] = df[use_types_gfa].replace(np.nan, 0)

    x_max = df[use_types_gfa].values.max()
    x_min = df[use_types_gfa].values.min()

    for (i,line), gfa in zip(enumerate(df[use_types].values), df[use_types_gfa].values):
        # Clés sans doublons
        line_k = np.unique(np.array(list(filter(lambda x: x==x, line))))
        # On met les valeurs de gfa pour ces clés
        use_types_gfa_df.loc[i,[f"{x}_ut" for x in line_k]] = gfa[:len(line_k)]

    use_types_gfa_df = (use_types_gfa_df - x_min) / (x_max - x_min)
    
    df = df.drop(columns=use_types+use_types_gfa)
    df[use_types_gfa_df.columns] = use_types_gfa_df.replace(np.nan, 0)

    return df

encode_use_types(df).head()

On doit faire attention à appliquer un scaling sur les données de GFA dans l'ensemble avant de faire cet encodage.

En effet si on ne fait pas ça le scaling s'effectuera sur les UseTypes de façon indépendantes.

In [ ]:
# sns.pairplot(data=df[["Latitude", "Longitude", "PropertyGFABuilding(s)", "PropertyGFAParking", "LargestPropertyUseTypeGFA", "SecondLargestPropertyUseTypeGFA", "ThirdLargestPropertyUseTypeGFA"]], kind="kde")

In [ ]:
targets = ["TotalGHGEmissions", "SiteEnergyUse(kBtu)"]
ohe_cols = ["BuildingType", "ZipCode", "CouncilDistrictCode", "Neighborhood", "YearBuiltRange", "Parking"]
std_numerical_cols = ["Latitude", "Longitude", "PropertyGFABuilding(s)", "PropertyGFAParking"] + targets

In [ ]:
from sklearn.preprocessing import MinMaxScaler

def encoding(df, categorical_cols, std_cols, suffix="_std"):
    # Scaling
    for col in std_cols:
        df[f"{col}{suffix}"] = (df[col] - df[col].mean()) / df[col].std()
        df = df.drop(columns=col)

    # Use types processing
    df = encode_use_types(df, use_types_gfa=use_types_gfa)

    # OneHotEncoding
    for col in categorical_cols:
        dummies = pd.get_dummies(df[col]).replace(np.nan, 0)
        df = pd.concat([df, dummies], axis=1).drop(columns=col)

    return df

encoding(df, ohe_cols, std_numerical_cols)

Magnifique !

In [ ]:
from sklearn.model_selection import train_test_split


def preprocessing(df, targets, categorical_cols, std_cols, random_state):
    df = encoding(df, categorical_cols, std_cols)

    targets = [f"{x}_std" for x in targets]

    target_splits = []
    X = df.drop(columns=targets)
    names = np.array([str(x) for x in X.columns])
    X = X.values
    for target in targets:
        y = df[target].values.ravel()

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state)

        print(f"Target {target} with {X.shape[1]} features ({y.shape[0]} lines). Train={X_train.shape[0]} ; Test={X_test.shape[0]}")

        target_splits.append([X_train, X_test, y_train, y_test])

    return target_splits, names

In [ ]:
random_state = 0
(emissionTTS, energyTTS), feature_names = preprocessing(df, targets, ohe_cols, std_numerical_cols, random_state)

In [ ]:
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error, median_absolute_error, r2_score
from sklearn.model_selection import cross_val_score

model = DummyRegressor()

def evaluation(model, tts, cv=5, verbose=0):
    X_train, X_test, y_train, y_test = tts

    r2s = cross_val_score(model, X_train, y_train, cv=cv, scoring="r2")

    if(verbose):
        print(" "*7 + " Validation set metrics")
        print(f"{'R2': <6} : {r2s.mean():.4f}")

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    test_r2 = r2_score(y_test, y_pred)
    if(verbose):
        print(" "*7 + " Test set metrics")
        print(f"{'RMSE': <6} : {mean_squared_error(y_test, y_pred, squared=False):.2f}")
        print(f"{'MAE': <6} : {median_absolute_error(y_test, y_pred):.2f}")
        print(f"{'R2': <6} : {test_r2:.4f}")
    return r2s, test_r2

evaluation(model, emissionTTS, verbose=1);

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import (
    RandomForestRegressor,
    GradientBoostingRegressor,
    BaggingRegressor
)
from sklearn.linear_model import RidgeCV, LassoCV, ElasticNetCV
from tqdm import tqdm
from sklearn.model_selection import BaseCrossValidator

def compare_models(tts, cv=5):
    models = [
        SVR(C=500, epsilon=0.001, tol=1e-1),
        # MLPRegressor((150,100, 80, 50), random_state=random_state, alpha=0.001, max_iter=500), # Using this for testing only but not in the scope of this project
        RidgeCV(alphas=np.linspace(0.01, 1000, 50)),
        RandomForestRegressor(random_state=random_state),
        ElasticNetCV(random_state=random_state),
        GradientBoostingRegressor(random_state=random_state),
        DecisionTreeRegressor(random_state=random_state),
        KNeighborsRegressor(),
        BaggingRegressor(n_estimators=100),
        LassoCV(tol=0.01, random_state=random_state)
    ]

    val_keys = []
    if isinstance(cv, BaseCrossValidator):
        print("ISTANCE")
        val_keys = [f"val_r2_{i+1}" for (i) in range(cv.get_n_splits())]
    else:
        val_keys = [f"val_r2_{i+1}" for (i) in range(cv)]
    

    metrics = pd.DataFrame(columns=["name"] + val_keys + ["val_r2", "test_r2"])

    for model in tqdm(models):
        name = type(model).__name__
        vals_r2, test_r2 = evaluation(model, tts, cv=cv)
        metrics.loc[len(metrics)] = {"name": name} | {k:v for (k,v) in zip(val_keys, vals_r2)} | {"val_r2" : vals_r2.mean(), "test_r2": test_r2}

    return metrics, models

In [ ]:
emission_metrics, emission_models = compare_models(emissionTTS)
emission_metrics

In [ ]:
energy_metrics, energy_models = compare_models(energyTTS)
energy_metrics

In [ ]:
def show_model_perfs(target_vals):
    _, axs = plt.subplots(1, 2, figsize=(16,4))
    for i, v_type in enumerate(["val_r2", "test_r2"]):
        sns.scatterplot(data=target_vals[0], x="name", y=v_type, ax=axs[i])
        sns.scatterplot(data=target_vals[1], x="name", y=v_type, ax=axs[i])
        axs[i].set_xticklabels(energy_metrics.name, rotation=45, ha="right")
        axs[i].set_title(f"Models {v_type} on targets")
        axs[i].legend(targets)
    plt.show()

show_model_perfs([emission_metrics, energy_metrics])

Globalement notre jeu de donnée est plus pertinent pour effectuer des prédictions sur TotalGHGEmissions. On remarque que les 2 targets sont inversées au niveau des résultats entre le validation set et le test test.

- TotalGHGEmission est plus bas que SiteEnergyUse sur le validation set que sur le test set
- SiteEnergyUse est plus bas que TotalGHGEmission sur le test set que sur le validation set

Il y a certainement des variables moins pertinentes pour la prédiction de SiteEnergyUse(kBtu).

Analysons les meilleurs modèles

In [ ]:
import shap
shap.initjs()

shap_models = [emission_models[1], emission_models[-1], emission_models[3], emission_models[4]]

emissions = emissionTTS[0][:1000]

for model in shap_models:
    print(type(model).__name__)
    evaluation(model, emissionTTS)
    explainer = shap.Explainer(model.predict, masker=emissions, feature_names=feature_names)
    shap_values = explainer(emissions)
    
    shap.plots.bar(shap_values, max_display=20)

On remarque que la surface est bien majoritairement utilisée par nos modèles pour effectuer nos prédictions. Peu importe le modèle parmis les 3 présentés.

Mais la feature la plus représentée est celle concernant les hôpitaux. Ainsi, si le bâtiment est un hopital, alors sa consommation sera très élevée.

- RidgeCV conserve beaucoup d'autre features, ce qui est logique. RidgeCV Regroupe les features par similarité
- LassoCV lui cherche à annuler les features qui ne l'interesse pas, c'est ce qu'on voit sur son analyse shap
- L'ElasticNet lui est un entre 2 qui prends en compte les 2 régularisations l1 et l2 (Ridge et Lasso), ce que l'on peut voir sur l'analyse
- GradientBoostingRegressor va incrémentalement chercher à optimiser un modèle

D'après l'analyse on a peu de lignes avec des hopitaux mais celles qui existent ont des valeurs de GFA élevées. Or ce n'est pas la seule catégorie comme cela. Il y a aussi "Courthouse" et "Convention Center" par exemple.

Nos premiers modèles peuvent donc être améliorés en sélectionnant / en regroupant les bonnes features.

In [ ]:
# keeping best models
target_models = [
    (RidgeCV(alphas=np.linspace(0.01, 1000, 20), scoring='r2'), {"cv": np.arange(3, 10)}),
    (LassoCV(alphas=np.linspace(0.01, 1000, 20), random_state=random_state), {"tol": np.linspace(1e-4, 1e-2, 4)}),
    (ElasticNetCV(alphas=np.linspace(0.01, 1000, 20), random_state=random_state), {}),
    (GradientBoostingRegressor(loss="squared_error", random_state=random_state), {"learning_rate": np.linspace(1e-3, 1e-1, 4), "max_depth": np.linspace(3, 15, 4).astype("int")})
]

target_tts_model = list(zip(targets, [emissionTTS, energyTTS], [target_models, target_models]))

In [ ]:
for target, tts, models_params in target_tts_model:
    print("="*5 + f"> {target}")
    # show r2's
    for model,_ in models_params:
        print(type(model).__name__)
        evaluation(model, tts, verbose=True);

On peut déjà effectuer un petit GridSearch pour essayer d'améliorer nos performances et voir s'il y a une différence.

## Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV

def search_cv(model, tts, params):
    X_train, _, y_train, __ = tts

    cv = GridSearchCV(
        model,
        cv=3,
        param_grid=params,
        scoring=["r2", "neg_root_mean_squared_error", "neg_median_absolute_error"],
        n_jobs=-1,
        refit="r2"
    )
    cv.fit(X_train, y_train)
    return cv

cvs = []

for target, tts, model_params in target_tts_model:
    print("GridSearchs on " + target + " :")
    for model, params in tqdm(model_params):
        df_cv = search_cv(model, tts, params)
        cvs.append(df_cv)

In [ ]:
# TotalGHGEmissions - Ridge
pd.DataFrame(cvs[0].cv_results_)

In [ ]:
# TotalGHGEmissions - Lasso
pd.DataFrame(cvs[1].cv_results_)

In [ ]:
# TotalGHGEmissions - Elastic
pd.DataFrame(cvs[2].cv_results_)

In [ ]:
# TotalGHGEmissions - GradientBoosting
pd.DataFrame(cvs[3].cv_results_)

### GridSearch - TotalGHGEmissions

On remarque que les splits ont des valeurs de r2 très différentes. C'est certainement car il y a une présence de valeurs un peu grandes dans ceux-ci qui font que quand on passe sur le validation set nos modèle ont du mal à effectuer les prédictions.

On remarque que c'est le cas à chaque fois pour le split 2 (le dernier split)

On peut éventuellement :
- Analyser les données qui mènent à cette erreur
- Effectuer des splits avec une meilleure stratification que pour le train test split (stratify=True)
- Modifier le nombre de cross validations

In [ ]:
# SiteEnergyUse - Ridge
pd.DataFrame(cvs[5].cv_results_)

In [ ]:
# SiteEnergyUse - Lasso
pd.DataFrame(cvs[6].cv_results_)

In [ ]:
# SiteEnergyUse - Elastic
pd.DataFrame(cvs[7].cv_results_)

### GridSearch - TotalGHGEmissions

Le split 2 reste toujours un peu inférieur au 2 autres splits pour le score r2.

La median absolute error est différente et modifie le rang de nos grid search results. Ceci laisse entendre que nous avons parfois des erreurs importantes dans nos prédictions ce qui suppose la présence d'outliers.

# Itération 2

Certaines variables n'apporte pas beaucoup à notre modèle. On peut ainsi effectuer une réduction de dimension ce qui aura pour effet d'aider le modèle à apprendre sur des données plus cohérentes et également de réduire le temps d'entrainement.

Utilisons également les pipeline !

In [ ]:
df = data.copy()
df.head()

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer, MinMaxScaler
from sklearn import set_config

set_config(transform_output="pandas")

def encode_use_types(X):
    use_types_gfa_df = pd.DataFrame()
    # use type columns types/GFA
    use_types = [x for x in X.columns if "GFA" not in x]
    use_types_gfa = [x for x in X.columns if "GFA" in x]

    for (i,line), gfa in zip(enumerate(X[use_types].values), X[use_types_gfa].values):
        # Clés sans doublons
        line_k = np.unique(np.array(list(filter(lambda x: x==x, line))))
        # On met les valeurs de gfa pour ces clés
        use_types_gfa_df.loc[i,line_k] = gfa[:len(line_k)]


    use_types_gfa_df = use_types_gfa_df.fillna(0)
    use_types_gfa_df = MinMaxScaler().fit_transform(use_types_gfa_df)

    return use_types_gfa_df

preprocessing = Pipeline([
    ("encoder", ColumnTransformer([
                    ("standard_scaler", StandardScaler(), ["Latitude", "Longitude", "PropertyGFAParking", "PropertyGFABuilding(s)"]),
                    ("one_hot_encoded", OneHotEncoder(sparse_output=False), ["Parking", "YearBuiltRange", "Neighborhood", "CouncilDistrictCode", "ZipCode", "BuildingType"]),
                    ("use_types", FunctionTransformer(encode_use_types, validate=False), ["LargestPropertyUseType", "LargestPropertyUseTypeGFA", 'SecondLargestPropertyUseType', 'SecondLargestPropertyUseTypeGFA', 'ThirdLargestPropertyUseType', 'ThirdLargestPropertyUseTypeGFA'])
                ], remainder="drop")),
])

In [ ]:
X = df.loc[:, ~df.columns.isin(targets)]
y = df.loc[:, df.columns.isin(targets)]
y_emission = y.loc[:, targets[0]]
y_electricity = y.loc[:, targets[1]]

preprocessing.fit(X, y_emission)

In [ ]:
X_encoded = preprocessing.transform(X)
X_encoded

In [ ]:
X_encoded.describe()

In [ ]:
from sklearn.feature_selection import RFE

model = GradientBoostingRegressor()
rfe = RFE(model)
rfe.fit(X_encoded, y_emission)

In [ ]:
X_encoded.columns[rfe.get_support()].shape

In [ ]:
selection = Pipeline([
    ("preprocessing", preprocessing),
    ("rfe", RFE(GradientBoostingRegressor()))
])

In [ ]:
selection.fit(X, y_emission)

In [ ]:
X_encoded = selection.transform(X)
X_encoded

Afin de rompre l'incohérence entre le validation set et le test set on peut nous même effectuer une stratification cette fois-ci sur une autre variable que l'on considère importante, comme le property use type.

Ainsi on aura un peu de chaque type de bâtiments dans nos jeux de données.

Je ne peux pas encore effectuer une stratification sur ces données car je n'ai pas assez de property use types pour chaque valeur unique. En effet, parfois il y a une ligne avec un property use type unique. Ainsi je vais devoir effectuer un feature engineering pour rassembler certaines valeurs entre elles. Je me souviens qu'il y a la valeur "autre" sur laquelle je peux m'appuyer.

In [ ]:
df = data.copy()

In [ ]:
df.LargestPropertyUseType.unique().__len__()

In [ ]:
value_over = (df.LargestPropertyUseType.value_counts() > 5).reset_index()
value_over = value_over.loc[value_over.LargestPropertyUseType].set_index("index")
value_over.head()

In [ ]:
df.loc[~df.LargestPropertyUseType.isin(value_over.index), "LargestPropertyUseType"] = "Other"
df.LargestPropertyUseType.unique().__len__()

In [ ]:
df.loc[~df.SecondLargestPropertyUseType.isin(value_over.index), "SecondLargestPropertyUseType"] = "Other"
df.loc[~df.ThirdLargestPropertyUseType.isin(value_over.index), "ThirdLargestPropertyUseType"] = "Other"
(set(df.LargestPropertyUseType) | set(df.SecondLargestPropertyUseType) | set(df.ThirdLargestPropertyUseType)).__len__()

In [ ]:
order = df.LargestPropertyUseType.value_counts().sort_values(ascending=False).index

plt.figure(figsize=(18,6))
sns.countplot(data=df, x="LargestPropertyUseType", order=order)
plt.title("LargestPropertyUseType count")
plt.xticks(rotation=45, horizontalalignment="right");

In [ ]:
X = df.loc[:, ~df.columns.isin(targets)]

X_encoded = preprocessing.transform(X)
len([x for x in X_encoded.columns if "use_types__" in x])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_emission, test_size=0.3, random_state=random_state, stratify=df.LargestPropertyUseType)

In [ ]:
model.fit(X_train, y_train)
model.score(X_test, y_test)

In [ ]:
X_train_usetypes = df.iloc[X_train.index].LargestPropertyUseType
X_test_usetypes = df.iloc[X_test.index].LargestPropertyUseType

order = X_train_usetypes.unique()

fig, axs = plt.subplots(2, 1, figsize=(10,14), sharex=True)
sns.countplot(x=X_train_usetypes, ax=axs[0], order=order)
axs[0].set_ylabel("Train set stratification")
sns.countplot(x=X_test_usetypes, ax=axs[1], order=order)
axs[1].set_ylabel("Test set stratification")
plt.xticks(rotation=45, horizontalalignment="right");

In [ ]:
X_test.shape

In [ ]:
from sklearn.model_selection import StratifiedKFold

splits = 3

skf = StratifiedKFold(n_splits=splits, random_state=random_state, shuffle=True)

y_emission_discret = pd.cut(y_train, 20, labels=False)

folds = []
for i, (i_train, i_test) in enumerate(skf.split(df.iloc[X_train.index].LargestPropertyUseType, df.iloc[X_train.index].LargestPropertyUseType)):
    folds.append([i_train, i_test])


fig, axs = plt.subplots(splits, 1, sharex=True, sharey=True, figsize=(18,20))
for i, ax in enumerate(axs):
    property_fold = df.iloc[folds[i][1]]["LargestPropertyUseType"]
    sns.histplot(x=property_fold, kde=True, ax=ax, label=f'Fold-{i}')
    ax.set_ylabel('Frequency')
    if i == len(axs) - 1:
        ax.set_xlabel("PropertyUseTypes")
    ax.legend(frameon=False, handlelength=0)
plt.xticks(rotation=45, horizontalalignment="right")
plt.tight_layout()
plt.show()

model_scores = []
mean_model_scores = []
for model in tqdm(emission_models):
    scores = np.array([])
    for i_train, i_test in folds:
        Xtr, Xte, ytr, yte = X_train.iloc[i_train], X_train.iloc[i_test], y_train.iloc[i_train], y_train.iloc[i_test]

        model.fit(Xtr, ytr)
        y_pred = model.predict(Xte)

        scores = np.append(scores, r2_score(yte, y_pred))
    model_scores.append((type(model).__name__, scores))
    mean_model_scores.append(scores.mean())

[(name, score) for (name,_), score in zip(model_scores, mean_model_scores)]

La stratification a bien été effectuée en fonction du type de bâtiments. Peut être que maintenant nous auront de meilleurs résultats pour notre validation/test sets.

Or on remarque que les résultats sont pire qu'avant.

In [ ]:
test_scores = []
for model in tqdm(emission_models):
    y_pred = model.predict(X_test)
    test_scores.append(r2_score(y_test, y_pred))

[(name, score) for (name,_), score in zip(model_scores, test_scores)]

On a toujours un grand écart entre le validation set et le test test. Les prédictions sur le test set sont étrangement bonnes.

Si on enlève le random_state on remarque que pour certains splits notre modèle est très fort lors de la prédiction sur notre test set. Ceci renforce le fait que les outliers faussent nos prédiction.

C'était une bonne idée de les conserver pour pouvoir prédire avec des données d'outliers, mais si l'outlier déroute nos modèle alors autant s'en débarasser.

Essayons de voir pourquoi nos modèles sont si mauvais.

In [ ]:
from IPython.core.display import HTML

shap.initjs()

model = emission_models[4]

exp = shap.Explainer(model, feature_names=X_encoded.columns)

shap_values = exp.shap_values(X_test)

p = shap.plots.force(exp.expected_value[0], shap_values, X_test)

HTML(f"<div style='width:100%;background-color:white'>{p.html()}</div>")

In [ ]:
shap.summary_plot(shap_values, X_test,  plot_size=[18,10])

On peut clairement conclure que certaines lignes ont plus de poids pour notre modèle. Essayons de regarder de quelles lignes il s'agit pour en savoir plus.

In [ ]:
expected = exp.expected_value[0]
expected

In [ ]:
shap_values = pd.DataFrame(shap_values, columns=X_encoded.columns)
shap_values

In [ ]:
shap_outliers = df.iloc[X_test.index].loc[(shap_values > 500).any(axis=1).values]
shap_outliers

In [ ]:
df.iloc[X_test.index].describe()

In [ ]:
sns.scatterplot(data=df, x="PropertyGFABuilding(s)", y="TotalGHGEmissions", hue=df.index.isin(shap_outliers.index))
plt.title("PropertyBuildingGFA by TotalGHGEmissions colored by shap outlier");

Déjà 2 de nos shaply values sont issues de d'outliers. Les autres sont éparpillées à l'intérieur des autres données.

In [ ]:
plt.figure(figsize=(16, 6))
sns.scatterplot(data=df, x="LargestPropertyUseType", y="TotalGHGEmissions", hue=df.index.isin(shap_outliers.index))
plt.xticks(rotation=45, horizontalalignment="right");

In [ ]:
to_delete_index = df[(df.LargestPropertyUseType == "Hospital (General Medical & Surgical)") & (df.TotalGHGEmissions > 1e4)].index
to_delete_index = np.concatenate((to_delete_index, df[(df.LargestPropertyUseType == "Medical Office") & (df.TotalGHGEmissions > 2e3)].index))
to_delete_index

Nous sommes uniquement sur le test set. Voyons voir le training set.

In [ ]:
exp = shap.Explainer(model, feature_names=X_encoded.columns)

shap_values = exp.shap_values(X_train)

In [ ]:
shap.summary_plot(shap_values, X_train,  plot_size=[18,10])

In [ ]:
shap_outliers = df.iloc[X_train.index].loc[(shap_values > 1.5e3).any(axis=1)]
shap_outliers

In [ ]:
df.describe()

Les shap_outliers sont au dessus de la moyenne pour PropertyGFABuilding(s), TotalGHGEmissions et SiteEnergyUse(kBtu). Ce n'est pas une coïncidence. On va donc retirer ces outliers pour éviter d'entrainer dessus.

In [ ]:
df = df[~df.index.isin(np.concatenate((to_delete_index, shap_outliers.index)))]
df = df.reset_index(drop=True)

In [ ]:
preprocessing = Pipeline([
    ("encoder", ColumnTransformer([
                    ("standard_scaler", StandardScaler(), ["Latitude", "Longitude", "PropertyGFAParking", "PropertyGFABuilding(s)"]),
                    ("one_hot_encoded", OneHotEncoder(sparse_output=False), ["Parking", "YearBuiltRange", "Neighborhood", "CouncilDistrictCode", "ZipCode", "BuildingType"]),
                    ("use_types", FunctionTransformer(encode_use_types, validate=False), ["LargestPropertyUseType", "LargestPropertyUseTypeGFA", 'SecondLargestPropertyUseType', 'SecondLargestPropertyUseTypeGFA', 'ThirdLargestPropertyUseType', 'ThirdLargestPropertyUseTypeGFA'])
                ], remainder="drop")),
])

In [ ]:

X = df.loc[:, ~df.columns.isin(targets)]
y = df.loc[:, df.columns.isin(targets)]
y_emission = y.loc[:, targets[0]]
y_electricity = y.loc[:, targets[1]]

X_encoded = preprocessing.fit_transform(X, y_emission)
X_encoded

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_emission, test_size=0.3, random_state=random_state)

splits = 3

skf = StratifiedKFold(n_splits=splits, random_state=random_state, shuffle=True)

y_emission_discret = pd.cut(y_train, 50, labels=False)

folds = []
for i, (i_train, i_test) in enumerate(skf.split(y_emission_discret, y_emission_discret)):
    folds.append([i_train, i_test])

model_scores = []
mean_model_scores = []
for model in tqdm(emission_models):
    scores = np.array([])
    for i_train, i_test in folds:
        Xtr, Xte, ytr, yte = X_train.iloc[i_train], X_train.iloc[i_test], y_train.iloc[i_train], y_train.iloc[i_test]

        model.fit(Xtr, ytr)
        y_pred = model.predict(Xte)

        scores = np.append(scores, r2_score(yte, y_pred))
    model_scores.append((type(model).__name__, scores))
    mean_model_scores.append(scores.mean())

[(name, score) for (name,_), score in zip(model_scores, mean_model_scores)]

Les résultats sont vraiment horribles. Revenont sur nos pas.

# Itération 3

On a appris de l'itération 2 que :

- Une stratification par le PropertyUseType n'est pas une bonne solution
- On a en effet la présence d'outliers

D'après ceux-ci, on peut essayer de :

- Filtrer les outliers encore plus qu'avant, si un souhaite qu'un outlier puisse être détecté par notre modèle il faut plus de données qui lui ressemble. On peut également opter pour une data augmentation
- Remettre en question l'utilité du PropertyUseType. Il faut certainement créer des groupes d'émetteurs comme "Gros Emetteur", "Moyen Emetteur", "Petit Emetteur", ainsi aboutir à une réduction de dimensionalité

Filtrons d'abord les outliers et observons s'il y a des changements

In [ ]:
df = data.copy()
df.head()

Observons les grosses erreurs qui sont effectuées sur le dataset

In [ ]:
preprocessing = Pipeline([
    ("encoder", ColumnTransformer([
                    ("standard_scaler", StandardScaler(), ["Latitude", "Longitude", "PropertyGFAParking", "PropertyGFABuilding(s)"]),
                    ("one_hot_encoded", OneHotEncoder(sparse_output=False), ["Parking", "YearBuiltRange", "Neighborhood", "CouncilDistrictCode", "ZipCode", "BuildingType"]),
                    ("use_types", FunctionTransformer(encode_use_types, validate=False), ["LargestPropertyUseType", "LargestPropertyUseTypeGFA", 'SecondLargestPropertyUseType', 'SecondLargestPropertyUseTypeGFA', 'ThirdLargestPropertyUseType', 'ThirdLargestPropertyUseTypeGFA'])
                ], remainder="drop")),
])

In [ ]:
X = df.loc[:, ~df.columns.isin(targets)]
y = df.loc[:, df.columns.isin(targets)]
y_emission = y.loc[:, targets[0]]
y_electricity = y.loc[:, targets[1]]

X_encoded = preprocessing.fit_transform(X, y_emission)
X_encoded.head()

In [ ]:
tts_emission = train_test_split(X_encoded, y_emission, test_size=0.3, random_state=random_state)

model_metrics, emission_models = compare_models(tts_emission, cv=3)
model_metrics

In [ ]:
model = emission_models[4]
model

In [ ]:
model.fit(tts[0], tts[2])

y_pred = model.predict(X_encoded.values)

In [ ]:
r2_score(y_emission, abs(y_pred))

In [ ]:
mean_squared_error(y_emission, y_pred, squared=False)

In [ ]:
mae = median_absolute_error(y_emission, y_pred)
mae

Observons les pires prédictions (supérieur à la mae)

In [ ]:
(np.array(y_pred) < 0).nonzero()

In [ ]:
over_mae_i = (y_pred > mae).nonzero()
over_mae_i

In [ ]:
df.iloc[(np.array(y_pred) < 0).nonzero()]

In [ ]:
df.describe()